# Scraping Data Rescue Project
github: https://github.com/datarescueproject/portal/

In [42]:
# Imports
import re
import requests
import yaml
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from urllib.parse import urlparse

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)



Getting info from MD files

In [2]:
owner = "datarescueproject"
repo = "portal"
repo_url = f"https://github.com/datarescueproject/portal/"
folder = "_datasets/"
branch = 'main'

In [3]:
ref = requests.get(f"https://api.github.com/repos/datarescueproject/portal/git/refs/heads/{branch}").json()
commit_sha = ref['object']['sha']
tree_sha = requests.get(f"https://api.github.com/repos/datarescueproject/portal/git/commits/{commit_sha}").json()["tree"]["sha"]
tree = requests.get(f"https://api.github.com/repos/datarescueproject/portal/git/trees/{tree_sha}", params={"recursive":"1"}).json()["tree"]

In [4]:
md_files = []
for item in tree:
    if item["type"] != "blob":
        continue
    if not item["path"].startswith(folder):
        continue
    if not item["path"].lower().endswith(".md"):
        continue
    md_files.append(item["path"])

md_files = sorted(md_files)
print(len(md_files))

1717


In [5]:
frontmatter_regex = re.compile(
    r'^\s*---\s*\n(.*?)\n---\s*\n?',
    re.DOTALL
)

projects = []
resources = []

for path in md_files:
    raw_url = f'https://raw.githubusercontent.com/datarescueproject/portal/main/{path}'
    markdown = requests.get(raw_url).text

    match = frontmatter_regex.match(markdown)
    if not match:
        frontmatter = {}
    else:
        frontmatter = yaml.safe_load(match.group(1)) or {}

    if not isinstance(frontmatter, dict):
        frontmatter = {}

    project_row = {
        'file': path,
        'schema': frontmatter.get('schema'),
        'title': frontmatter.get('title'),
        'organization': frontmatter.get('organization'),
        'agency': frontmatter.get('agency'),
        'websites': frontmatter.get('websites'),
        'data_source': frontmatter.get('data_source'),
        'description': frontmatter.get('description'),
        'last_modified': frontmatter.get('last_modified'),
        'metadata_available': frontmatter.get('metadata_available'),
        'metadata_url': frontmatter.get('metadata_url'),
    }

    categories = frontmatter.get('category', [])
    if isinstance(categories, list):
        project_row['category'] = '; '.join(map(str, categories))
    elif categories:
        project_row['category'] = str(categories)
    else:
        project_row['category'] = ''

    resource_list = frontmatter.get('resources', [])
    if not isinstance(resource_list, list):
        resource_list = []

    project_row['resource_count'] = len(resource_list)
    projects.append(project_row)

    for res in resource_list:
        if not isinstance(res, dict):
            continue

        resources.append({
            'file': path,
            'project_title': frontmatter.get('title'),
            'id': res.get('id'),
            'url': res.get('url'),
            'format': res.get('format'),
            'status': res.get('status'),
            'size': res.get('size'),
            'download_date': res.get('download_date'),
            'maintainer': res.get('maintainer'),
            'notes': res.get('notes'),
        })

projects_df = pd.DataFrame(projects)
resources_df = pd.DataFrame(resources)

projects_df.head()

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count
0,_datasets/10j-injunctions.md,data_rescue_project,10(j) Injunctions,National Labor Relations Board,National Labor Relations Board,nlrb.gov,https://www.nlrb.gov/what-we-do/investigate-ch...,None,2025-04-20,True,None,Labor & Employment,1
1,_datasets/2002-2024-mbda-grantees.md,data_rescue_project,2002-2024 MBDA Grantees,Minority Business Development Agency,Department of Commerce,mbda.gov,https://www.mbda.gov/research/data/mbda-grantees,None,2025-04-02,False,None,Business & Economy,1
2,_datasets/2006-iur-public-database.md,data_rescue_project,2006 IUR Public Database,Environmental Protection Agency,Environmental Protection Agency,epa.gov,https://www.epa.gov/chemical-data-reporting/do...,None,2025-09-18,True,https://www.epa.gov/chemical-data-reporting/do...,Climate & Environment,1
3,_datasets/2014-minority-veteran-report.md,data_rescue_project,2014 Minority Veteran Report,Office of Information and Technology - IT Oper...,Department of Veterans Affairs,data.va.gov,https://www.data.va.gov/stories/s/gavm-n6bm,None,2025-03-17,False,None,Science & Research; Military & Veterans Affairs,1
4,_datasets/2016-americorps-mes-americorps-membe...,data_rescue_project,2016 AmeriCorps MES AmeriCorps Member Exit Survey,AmeriCorps,AmeriCorps,data.americorps.gov,https://data.americorps.gov/National-Service/2...,None,2025-03-10,False,None,Military & Veterans Affairs; Humanitarian & Di...,1


In [ ]:
projects_df.to_csv('datarescuetracker.csv')
resources_df.to_csv('resources.csv')

In [6]:
projects_df

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count
0,_datasets/10j-injunctions.md,data_rescue_project,10(j) Injunctions,National Labor Relations Board,National Labor Relations Board,nlrb.gov,https://www.nlrb.gov/what-we-do/investigate-ch...,None,2025-04-20,True,None,Labor & Employment,1
1,_datasets/2002-2024-mbda-grantees.md,data_rescue_project,2002-2024 MBDA Grantees,Minority Business Development Agency,Department of Commerce,mbda.gov,https://www.mbda.gov/research/data/mbda-grantees,None,2025-04-02,False,None,Business & Economy,1
2,_datasets/2006-iur-public-database.md,data_rescue_project,2006 IUR Public Database,Environmental Protection Agency,Environmental Protection Agency,epa.gov,https://www.epa.gov/chemical-data-reporting/do...,None,2025-09-18,True,https://www.epa.gov/chemical-data-reporting/do...,Climate & Environment,1
3,_datasets/2014-minority-veteran-report.md,data_rescue_project,2014 Minority Veteran Report,Office of Information and Technology - IT Oper...,Department of Veterans Affairs,data.va.gov,https://www.data.va.gov/stories/s/gavm-n6bm,None,2025-03-17,False,None,Science & Research; Military & Veterans Affairs,1
4,_datasets/2016-americorps-mes-americorps-membe...,data_rescue_project,2016 AmeriCorps MES AmeriCorps Member Exit Survey,AmeriCorps,AmeriCorps,data.americorps.gov,https://data.americorps.gov/National-Service/2...,None,2025-03-10,False,None,Military & Veterans Affairs; Humanitarian & Di...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712,_datasets/yemen-usaid-education-data.md,data_rescue_project,Yemen USAID Education Data,M/CIO Development Information Solution (DIS),U.S. Agency for International Development,data.usaid.gov,data.usaid.gov,None,2025-11-03,False,None,Education; Humanitarian & Disaster Relief,1
1713,_datasets/your-right-to-federal-records.md,data_rescue_project,Your Right to Federal Records,Federal Mediation and Conciliation Service,Federal Mediation and Conciliation Service,fmcs.gov,https://www.fmcs.gov/resources/documents-and-d...,None,2025-03-31,False,None,Labor & Employment,1
1714,_datasets/youth-risk-behavior-surveillance-sys...,data_rescue_project,Youth Risk Behavior Surveillance System (YRBSS...,Centers for Disease Control and Prevention (CDC),Department of Health and Human Services,cdc.gov,https://www.cdc.gov/yrbs/about/index.html,None,2025-11-05,False,None,Health & Healthcare,1
1715,_datasets/youth-risk-behavior-surveillance-sys...,data_rescue_project,Youth Risk Behavior Surveillance System (YRBSS),Centers for Disease Control and Prevention (CDC),Department of Health and Human Services,cdc.gov,https://www.cdc.gov/index.html,None,2025-05-19,True,None,Health & Healthcare,2


In [7]:
resources_df

,file,project_title,id,url,format,status,size,download_date,maintainer,notes
0,_datasets/10j-injunctions.md,10(j) Injunctions,760,https://doi.org/10.3886/E226824V1,"CSV, TXT",Finished,0.000,2025-04-07,"DRP, DL",Section 10(j) of the National Labor Relations ...
1,_datasets/2002-2024-mbda-grantees.md,2002-2024 MBDA Grantees,697,https://www.datalumos.org/datalumos/project/22...,CSV,Finished,0.000,2025-03-18,"DRP, DL",None
2,_datasets/2006-iur-public-database.md,2006 IUR Public Database,1281,https://dataverse.harvard.edu/dataset.xhtml?pe...,"mhtml, ZIP, PDF",Finished,0.005,2025-02-26,"HD, CAFE-RCC",There was a 2006 version and a 2002-1986 versi...
3,_datasets/2014-minority-veteran-report.md,2014 Minority Veteran Report,412,https://www.datalumos.org/datalumos/project/22...,CSV,Finished,0.000,2025-03-11,"DRP, DL",None
4,_datasets/2016-americorps-mes-americorps-membe...,2016 AmeriCorps MES AmeriCorps Member Exit Survey,396,https://www.datalumos.org/datalumos/project/22...,"CSV, XLSX, PDF",Finished,0.000,2025-03-08,"DRP, DL",None
...,...,...,...,...,...,...,...,...,...,...
1760,_datasets/your-right-to-federal-records.md,Your Right to Federal Records,644,https://www.datalumos.org/datalumos/project/22...,PDF,Finished,0.000,2025-03-15,"DRP, DL",Regulations and Policies Tab
1761,_datasets/youth-risk-behavior-surveillance-sys...,Youth Risk Behavior Surveillance System (YRBSS...,1503,https://www.datalumos.org/datalumos/project/22...,None,Finished,NaN,None,DL,None
1762,_datasets/youth-risk-behavior-surveillance-sys...,Youth Risk Behavior Surveillance System (YRBSS),326,https://www.dropbox.com/scl/fo/2t0ehrteq62jqrj...,"DTA, XLSX, do, MDB, dat, SPS, SAS",Finished,0.000,2025-01-31,ICPSR,None
1763,_datasets/youth-risk-behavior-surveillance-sys...,Youth Risk Behavior Surveillance System (YRBSS),1041,https://urban-data-catalog.s3.us-east-1.amazon...,ZIP,Finished,0.877,2025-02-05,UI,The Youth Risk Behavior Surveillance System fo...


In [8]:
df = pd.merge(projects_df, resources_df, on='file')

In [9]:
df

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,...,resource_count,project_title,id,url,format,status,size,download_date,maintainer,notes
0,_datasets/10j-injunctions.md,data_rescue_project,10(j) Injunctions,National Labor Relations Board,National Labor Relations Board,nlrb.gov,https://www.nlrb.gov/what-we-do/investigate-ch...,None,2025-04-20,True,...,1,10(j) Injunctions,760,https://doi.org/10.3886/E226824V1,"CSV, TXT",Finished,0.000,2025-04-07,"DRP, DL",Section 10(j) of the National Labor Relations ...
1,_datasets/2002-2024-mbda-grantees.md,data_rescue_project,2002-2024 MBDA Grantees,Minority Business Development Agency,Department of Commerce,mbda.gov,https://www.mbda.gov/research/data/mbda-grantees,None,2025-04-02,False,...,1,2002-2024 MBDA Grantees,697,https://www.datalumos.org/datalumos/project/22...,CSV,Finished,0.000,2025-03-18,"DRP, DL",None
2,_datasets/2006-iur-public-database.md,data_rescue_project,2006 IUR Public Database,Environmental Protection Agency,Environmental Protection Agency,epa.gov,https://www.epa.gov/chemical-data-reporting/do...,None,2025-09-18,True,...,1,2006 IUR Public Database,1281,https://dataverse.harvard.edu/dataset.xhtml?pe...,"mhtml, ZIP, PDF",Finished,0.005,2025-02-26,"HD, CAFE-RCC",There was a 2006 version and a 2002-1986 versi...
3,_datasets/2014-minority-veteran-report.md,data_rescue_project,2014 Minority Veteran Report,Office of Information and Technology - IT Oper...,Department of Veterans Affairs,data.va.gov,https://www.data.va.gov/stories/s/gavm-n6bm,None,2025-03-17,False,...,1,2014 Minority Veteran Report,412,https://www.datalumos.org/datalumos/project/22...,CSV,Finished,0.000,2025-03-11,"DRP, DL",None
4,_datasets/2016-americorps-mes-americorps-membe...,data_rescue_project,2016 AmeriCorps MES AmeriCorps Member Exit Survey,AmeriCorps,AmeriCorps,data.americorps.gov,https://data.americorps.gov/National-Service/2...,None,2025-03-10,False,...,1,2016 AmeriCorps MES AmeriCorps Member Exit Survey,396,https://www.datalumos.org/datalumos/project/22...,"CSV, XLSX, PDF",Finished,0.000,2025-03-08,"DRP, DL",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1760,_datasets/your-right-to-federal-records.md,data_rescue_project,Your Right to Federal Records,Federal Mediation and Conciliation Service,Federal Mediation and Conciliation Service,fmcs.gov,https://www.fmcs.gov/resources/documents-and-d...,None,2025-03-31,False,...,1,Your Right to Federal Records,644,https://www.datalumos.org/datalumos/project/22...,PDF,Finished,0.000,2025-03-15,"DRP, DL",Regulations and Policies Tab
1761,_datasets/youth-risk-behavior-surveillance-sys...,data_rescue_project,Youth Risk Behavior Surveillance System (YRBSS...,Centers for Disease Control and Prevention (CDC),Department of Health and Human Services,cdc.gov,https://www.cdc.gov/yrbs/about/index.html,None,2025-11-05,False,...,1,Youth Risk Behavior Surveillance System (YRBSS...,1503,https://www.datalumos.org/datalumos/project/22...,None,Finished,NaN,None,DL,None
1762,_datasets/youth-risk-behavior-surveillance-sys...,data_rescue_project,Youth Risk Behavior Surveillance System (YRBSS),Centers for Disease Control and Prevention (CDC),Department of Health and Human Services,cdc.gov,https://www.cdc.gov/index.html,None,2025-05-19,True,...,2,Youth Risk Behavior Surveillance System (YRBSS),326,https://www.dropbox.com/scl/fo/2t0ehrteq62jqrj...,"DTA, XLSX, do, MDB, dat, SPS, SAS",Finished,0.000,2025-01-31,ICPSR,None
1763,_datasets/youth-risk-behavior-surveillance-sys...,data_rescue_project,Youth Risk Behavior Surveillance System (YRBSS),Centers for Disease Control and Prevention (CDC),Department of Health and Human Services,cdc.gov,https://www.cdc.gov/index.html,None,2025-05-19,True,...,2,Youth Risk Behavior Surveillance System (YRBSS),1041,https://urban-data-catalog.s3.us-east-1.amazon...,ZIP,Finished,0.877,2025-02-05,UI,The Youth Risk Behavior Surveillance System fo...


extracting domains

In [11]:
def get_domain(url):
    if not isinstance(url, str):
        return None
    return urlparse(url).netloc

In [12]:
df["domain"] = df["url"].apply(get_domain)
df.head()

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,...,project_title,id,url,format,status,size,download_date,maintainer,notes,domain
0,_datasets/10j-injunctions.md,data_rescue_project,10(j) Injunctions,National Labor Relations Board,National Labor Relations Board,nlrb.gov,https://www.nlrb.gov/what-we-do/investigate-ch...,None,2025-04-20,True,...,10(j) Injunctions,760,https://doi.org/10.3886/E226824V1,"CSV, TXT",Finished,0.000,2025-04-07,"DRP, DL",Section 10(j) of the National Labor Relations ...,doi.org
1,_datasets/2002-2024-mbda-grantees.md,data_rescue_project,2002-2024 MBDA Grantees,Minority Business Development Agency,Department of Commerce,mbda.gov,https://www.mbda.gov/research/data/mbda-grantees,None,2025-04-02,False,...,2002-2024 MBDA Grantees,697,https://www.datalumos.org/datalumos/project/22...,CSV,Finished,0.000,2025-03-18,"DRP, DL",None,www.datalumos.org
2,_datasets/2006-iur-public-database.md,data_rescue_project,2006 IUR Public Database,Environmental Protection Agency,Environmental Protection Agency,epa.gov,https://www.epa.gov/chemical-data-reporting/do...,None,2025-09-18,True,...,2006 IUR Public Database,1281,https://dataverse.harvard.edu/dataset.xhtml?pe...,"mhtml, ZIP, PDF",Finished,0.005,2025-02-26,"HD, CAFE-RCC",There was a 2006 version and a 2002-1986 versi...,dataverse.harvard.edu
3,_datasets/2014-minority-veteran-report.md,data_rescue_project,2014 Minority Veteran Report,Office of Information and Technology - IT Oper...,Department of Veterans Affairs,data.va.gov,https://www.data.va.gov/stories/s/gavm-n6bm,None,2025-03-17,False,...,2014 Minority Veteran Report,412,https://www.datalumos.org/datalumos/project/22...,CSV,Finished,0.000,2025-03-11,"DRP, DL",None,www.datalumos.org
4,_datasets/2016-americorps-mes-americorps-membe...,data_rescue_project,2016 AmeriCorps MES AmeriCorps Member Exit Survey,AmeriCorps,AmeriCorps,data.americorps.gov,https://data.americorps.gov/National-Service/2...,None,2025-03-10,False,...,2016 AmeriCorps MES AmeriCorps Member Exit Survey,396,https://www.datalumos.org/datalumos/project/22...,"CSV, XLSX, PDF",Finished,0.000,2025-03-08,"DRP, DL",None,www.datalumos.org


In [13]:
df['domain'].value_counts()

domain
www.datalumos.org                                1331
www.dropbox.com                                    85
sciop.net                                          70
doi.org                                            48
dataverse.harvard.edu                              47
zenodo.org                                         34
archive.org                                        28
github.com                                         22
urban-data-catalog.s3.us-east-1.amazonaws.com      10
purl.stanford.edu                                  10
nlrbresearch.com                                    4
web.archive.org                                     3
arcgis.com                                          3
www.openicpsr.org                                   3
www.arcgis.com                                      3
biglocalnews.org                                    3
doi.pangaea.de                                      2
www.hydroshare.org                                  1
s3.amazonaws.com     

In [14]:
df[df['domain'].isna()]

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,...,project_title,id,url,format,status,size,download_date,maintainer,notes,domain
134,_datasets/bipartisan-infrastructure-law-and-in...,data_rescue_project,Bipartisan Infrastructure Law and Inflation Re...,National Oceanic and Atmospheric Administration,Department of Commerce,noaa.gov,https://www.noaa.gov/bil-ira-awards-explorer,None,2025-03-02,False,...,Bipartisan Infrastructure Law and Inflation Re...,211,None,CSV,Finished,0.000,None,"EDGI, ESRI",Local - EDGI; only downloaded the csv; not try...,None
135,_datasets/bls-downloads.md,data_rescue_project,BLS Downloads,Bureau of Labor Statistics,Department of Labor,download.bls.gov,https://download.bls.gov,None,2025-02-10,False,...,BLS Downloads,1,None,None,In Progress,47.000,2025-02-01,DRP,None,None
142,_datasets/building-code-adoption-tracking.md,data_rescue_project,Building Code Adoption Tracking,Federal Emergency Management Agency,Department of Homeland Security,fema.gov,https://stantec.maps.arcgis.com/apps/MapSeries...,None,2025-03-03,False,...,Building Code Adoption Tracking,205,None,"Interactive Map, PDF",In Progress,0.000,None,ESRI,Local - ESRI; Identify which building codes an...,None
150,_datasets/cdc-ftp-server.md,data_rescue_project,CDC FTP Server,Centers for Disease Control and Prevention (CDC),Department of Health and Human Services,ftp.cdc.gov,https://ftp.cdc.gov/,None,2025-05-23,False,...,CDC FTP Server,2,None,None,In Progress,213.000,2025-02-01,DRP,None,None
159,_datasets/census-bureau-ftp-server.md,data_rescue_project,Census Bureau FTP Server,Census Bureau,Department of Commerce,www2.census.gov,https://www2.census.gov,None,2025-05-23,False,...,Census Bureau FTP Server,3,None,"PDF, ZIP, XLS, HTML, TXT",Uploading,180.000,2025-02-01,DRP,Partial download.,None
160,_datasets/census-bureau-ftp-server.md,data_rescue_project,Census Bureau FTP Server,Census Bureau,Department of Commerce,www2.census.gov,https://www2.census.gov,None,2025-05-23,False,...,Census Bureau FTP Server,1051,None,"PDF, ZIP, XLS, HTML, TXT",Uploading,6200.000,2025-02-17,"DRP, ANON",Created as a torrent by Reddit user enchanting...,None
161,_datasets/census-bureau-ftp-server.md,data_rescue_project,Census Bureau FTP Server,Census Bureau,Department of Commerce,www2.census.gov,https://www2.census.gov,None,2025-05-23,False,...,Census Bureau FTP Server,1052,None,"PDF, ZIP, XLS, HTML, TXT",Uploading,1200.000,2025-02-13,"DRP, CREP",None,None
162,_datasets/census-bureau-ftp-server.md,data_rescue_project,Census Bureau FTP Server,Census Bureau,Department of Commerce,www2.census.gov,https://www2.census.gov,None,2025-05-23,False,...,Census Bureau FTP Server,1053,None,"PDF, ZIP, XLS, HTML, TXT",Uploading,1900.000,2025-02-20,DRP,None,None
188,_datasets/climate-indicators.md,data_rescue_project,Climate Indicators,Environmental Protection Agency,Environmental Protection Agency,epa.gov,https://www.epa.gov/climate-indicators,None,2025-03-02,False,...,Climate Indicators,186,None,None,Finished,0.000,None,ESRI,Local - ESRI; visual available on ArcGIS Onlin...,None
249,_datasets/consumer-financial-protection-bureau...,data_rescue_project,Consumer Financial Protection Bureau Website,Consumer Financial Protection Bureau,Consumer Financial Protection Bureau,consumerfinance.gov,https://files.consumerfinance.gov/,None,2025-02-20,False,...,Consumer Financial Protection Bureau Website,8,None,None,In Progress,0.000,2025-02-03,DRP,Can't find the link in internet archives and n...,None


Scraping datalumos for metadata (didnt work)

In [ ]:
!pip -q install playwright beautifulsoup4 lxml
!playwright install chromium

In [ ]:
doi_re = re.compile(r"(?:doi\.org/|doi:\s*)(10\.\d{4,9}/[-._;()/:A-Z0-9]+)", re.I)

mask = resources_df["domain"].isin(["www.datalumos.org", "datalumos.org"])
cols = ["principal_investigators","summary","project_title_page","geographic_coverage","time_periods","doi"]
resources_df.loc[mask, cols] = None

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()

    for i, row in resources_df[mask].iterrows():
        url = row["url"]
        print(url)
        try:
            await page.goto(url, wait_until="networkidle", timeout=60000)
            html = await page.content()
            soup = BeautifulSoup(html, "lxml")

            # scan all <strong> labels and capture the text in the same parent
            for strong in soup.find_all("strong"):
                label = strong.get_text(" ", strip=True)
                parent_text = strong.parent.get_text(" ", strip=True)
                value = re.sub(rf"^{re.escape(label)}\s*:?\s*", "", parent_text).strip()

                low = label.lower()
                if "principal investigator" in low:
                    resources_df.at[i, "principal_investigators"] = value
                elif low.startswith("summary"):
                    resources_df.at[i, "summary"] = value
                elif low.startswith("project title"):
                    resources_df.at[i, "project_title_page"] = value
                elif low.startswith("geographic coverage"):
                    resources_df.at[i, "geographic_coverage"] = value
                elif low.startswith("time period"):
                    resources_df.at[i, "time_periods"] = value

            text = soup.get_text(" ", strip=True)
            found = doi_re.findall(text)
            if found:
                resources_df.at[i, "doi"] = found[0]

        except Exception as e:
            print("Failed:", url, "|", e)

    await browser.close()

resources_df.loc[mask, ["url"] + cols].head()

In [ ]:
from playwright.async_api import async_playwright

test_url = resources_df.loc[mask, "url"].dropna().iloc[0]
print("Testing:", test_url)

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()
    await page.goto(test_url, timeout=60000)
    print(await page.title())
    await browser.close()

Filling in doi column for doi we already have

In [19]:
dois_provided = df['domain'] == 'doi.org'

In [20]:
doi_pattern = r"(10\.\d{4,9}/[-._;()/:A-Z0-9]+)"

df.loc[dois_provided, "doi"] = (
    df.loc[dois_provided, "url"]
    .str.extract(doi_pattern, expand=False)
    .str.lower()
)

In [24]:
df['doi'].isna().mean()

0.9762039660056657

Trying to get doi from urls for datalumos projects based n project number in the url

In [26]:
m = df["url"].astype(str).str.extract(r"/project/(\d+)/version/(V\d+)/", expand=True)
df["project_id"] = m[0]
df["version"] = m[1]

# guess DOI in the common ICPSR style: 10.3886/E{project_id}{version}
df["doi_guess"] = None
mask_dl = df["domain"].isin(["www.datalumos.org", "datalumos.org"]) & df["project_id"].notna() & df["version"].notna()
df.loc[mask_dl, "doi_guess"] = "10.3886/E" + df.loc[mask_dl, "project_id"] + df.loc[mask_dl, "version"]

df.loc[mask_dl, ["url","project_id","version","doi_guess"]]

,url,project_id,version,doi_guess
1,https://www.datalumos.org/datalumos/project/22...,223443,V1,10.3886/E223443V1
3,https://www.datalumos.org/datalumos/project/22...,222881,V1,10.3886/E222881V1
4,https://www.datalumos.org/datalumos/project/22...,222043,V1,10.3886/E222043V1
5,https://www.datalumos.org/datalumos/project/22...,222044,V2,10.3886/E222044V2
6,https://www.datalumos.org/datalumos/project/22...,222044,V2,10.3886/E222044V2
...,...,...,...,...
1749,https://www.datalumos.org/datalumos/project/22...,223962,V1,10.3886/E223962V1
1752,https://www.datalumos.org/datalumos/project/22...,222882,V1,10.3886/E222882V1
1753,https://www.datalumos.org/datalumos/project/22...,227293,V1,10.3886/E227293V1
1754,https://www.datalumos.org/datalumos/project/22...,227006,V1,10.3886/E227006V1


In [27]:
df['doi_guess'].value_counts()

doi_guess
10.3886/E222581V1    51
10.3886/E223141V1    48
10.3886/E229201V1    37
10.3886/E223001V1    30
10.3886/E224621V1    25
                     ..
10.3886/E219168V1     1
10.3886/E229742V2     1
10.3886/E219462V1     1
10.3886/E223523V1     1
10.3886/E227293V1     1
Name: count, Length: 710, dtype: int64

testing if it works

In [ ]:
test = "10.3886/E223443V1"

url = f"https://api.datacite.org/dois/{test}"

headers = {
    "Accept": "application/vnd.api+json"
}

response = requests.get(url, headers=headers)

print(response.text)

{"data":{"id":"10.3886/e223443v1","type":"dois","attributes":{"doi":"10.3886/e223443v1","prefix":"10.3886","suffix":"e223443v1","identifiers":[],"alternateIdentifiers":[],"creators":[{"name":"United States Department Of Commerce. Minority Business Development Agency","nameType":"Organizational","affiliation":[],"nameIdentifiers":[]}],"titles":[{"lang":"en","title":"2022-2024 MBDA Grantees"}],"publisher":"ICPSR - Interuniversity Consortium for Political and Social Research","container":{},"publicationYear":2025,"subjects":[{"lang":"en","subject":"grants"},{"lang":"en","subject":"minority businesses"}],"contributors":[],"dates":[{"date":"2022-01-01/2024-12-31","dateType":"Collected"},{"date":"2025","dateType":"Issued"}],"language":"en","types":{"ris":"DATA","bibtex":"misc","citeproc":"dataset","schemaOrg":"Dataset","resourceTypeGeneral":"Dataset"},"relatedIdentifiers":[{"relationType":"IsVersionOf","relatedIdentifier":"10.3886/e223443","relatedIdentifierType":"DOI"}],"relatedItems":[],"s

dois are correct, so fill in doi col with guesses

In [33]:
df["doi"] = df["doi"].combine_first(df["doi_guess"])

getting metadata from datacite

In [48]:
dois = df["doi"].dropna().astype(str).str.strip().str.lower().unique()
dois

array(['10.3886/e226824v1', '10.3886/e223443v1', '10.3886/e222881v1', ...,
       '10.3886/e238627v1', '10.3886/e220761v1', '10.3886/e237626v1'],
      dtype=object)

In [49]:
unique_dois = (
    df["doi"]
    .dropna()
    .astype(str)
    .str.strip()
    .str.lower()
    .unique()
)

len(unique_dois)

1057

also doi with no version

In [36]:
# grab project id
df["project_id"] = df["url"].astype(str).str.extract(r"/project/(\d+)", expand=False)

# grab version if present, otherwise default to V1 for datalumos project links
df["version"] = df["url"].astype(str).str.extract(r"/version/(V\d+)", expand=False)

mask_datalumos = df["domain"].isin(["www.datalumos.org", "datalumos.org"]) & df["project_id"].notna()
df.loc[mask_datalumos & df["version"].isna(), "version"] = "V1"

# build DOI guess for datalumos rows where we have project_id + version
mask_guess = mask_datalumos & df["version"].notna()
df.loc[mask_guess, "doi_guess"] = (
    "10.3886/E" + df.loc[mask_guess, "project_id"] + df.loc[mask_guess, "version"]
)

# (optional) if you want doi to be filled from doi_guess when missing
df["doi"] = df["doi"].fillna(df["doi_guess"])

df.loc[mask_datalumos, ["url", "project_id", "version", "doi_guess", "doi"]]

,url,project_id,version,doi_guess,doi
1,https://www.datalumos.org/datalumos/project/22...,223443,V1,10.3886/E223443V1,10.3886/E223443V1
3,https://www.datalumos.org/datalumos/project/22...,222881,V1,10.3886/E222881V1,10.3886/E222881V1
4,https://www.datalumos.org/datalumos/project/22...,222043,V1,10.3886/E222043V1,10.3886/E222043V1
5,https://www.datalumos.org/datalumos/project/22...,222044,V2,10.3886/E222044V2,10.3886/E222044V2
6,https://www.datalumos.org/datalumos/project/22...,222044,V2,10.3886/E222044V2,10.3886/E222044V2
...,...,...,...,...,...
1754,https://www.datalumos.org/datalumos/project/22...,227006,V1,10.3886/E227006V1,10.3886/E227006V1
1759,https://www.datalumos.org/datalumos/project/23...,238627,V1,10.3886/E238627V1,10.3886/E238627V1
1760,https://www.datalumos.org/datalumos/project/22...,223063,V1,10.3886/E223063V1,10.3886/E223063V1
1761,https://www.datalumos.org/datalumos/project/22...,220761,V1,10.3886/E220761V1,10.3886/E220761V1


lets see some duplicates

In [43]:
dup_rows = df[df["doi"].duplicated(keep=False)]
dup_rows[["doi", "url", "domain", 'title']].head(20)

,doi,url,domain,title
2,None,https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi10.7910/DVN/F3A62W,dataverse.harvard.edu,2006 IUR Public Database
4,10.3886/E222043V1,https://www.datalumos.org/datalumos/project/222043/version/V1/view,www.datalumos.org,2016 AmeriCorps MES AmeriCorps Member Exit Survey
5,10.3886/E222044V2,https://www.datalumos.org/datalumos/project/222044/version/V2/view,www.datalumos.org,2017-2023 CEV Findings National Rates of All Measures by Demographics from the Current Population Survey Civic Engagement and Volunteering Supplement
6,10.3886/E222044V2,https://www.datalumos.org/datalumos/project/222044/version/V2/view,www.datalumos.org,2017-2023 CEV Findings National Rates of All Measures from the Current Population Survey Civic Engagement and Volunteering Supplement
7,10.3886/E222044V2,https://www.datalumos.org/datalumos/project/222044/version/V2/view,www.datalumos.org,2017-2023 CEV Findings State-Level Rates of All Measures from the Current Population Survey Civic Engagement and Volunteering Supplement
8,10.3886/E222043V1,https://www.datalumos.org/datalumos/project/222043/version/V1/view,www.datalumos.org,2017 AmeriCorps MES AmeriCorps Member Exit Survey
10,10.3886/E222044V2,https://www.datalumos.org/datalumos/project/222044/version/V2/view,www.datalumos.org,2017 CEV Data Current Population Survey Civic Engagement and Volunteering Supplement
11,10.3886/E222043V1,https://www.datalumos.org/datalumos/project/222043/version/V1/view,www.datalumos.org,2018 AmeriCorps MES AmeriCorps Member Exit Survey
15,10.3886/E222043V1,https://www.datalumos.org/datalumos/project/222043/version/V1/view,www.datalumos.org,2019 AmeriCorps MES AmeriCorps Member Exit Survey
16,10.3886/E224361V1,https://www.datalumos.org/datalumos/project/224361/version/V1/view,www.datalumos.org,2019 CDFI Program Awardee Performance Data Snapshot


In [44]:
df['doi'].value_counts()

doi
10.3886/E222581V1    51
10.3886/E223141V1    48
10.3886/E229201V1    37
10.3886/E223001V1    30
10.3886/E224621V1    25
                     ..
10.3886/E239181V1     1
10.3886/E239227V1     1
10.3886/E239281V1     1
10.3886/E239225V1     1
10.3886/E237626V1     1
Name: count, Length: 1058, dtype: int64

In [45]:
df[df['doi'] == '10.3886/E222581V1']

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count,project_title,id,url,format,status,size,download_date,maintainer,notes,domain,doi,project_id,version,doi_guess
890,_datasets/medical-expenditure-panel-survey-meps-topics.md,data_rescue_project,Medical Expenditure Panel Survey (MEPS) Topics,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp,None,2025-03-18,False,None,Health & Healthcare,1,Medical Expenditure Panel Survey (MEPS) Topics,447,https://www.datalumos.org/datalumos/project/222581/version/V1/view,"TSV, ZIP",Finished,0.243,2025-03-03,"DRP, DL",None,www.datalumos.org,10.3886/E222581V1,222581,V1,10.3886/E222581V1
904,_datasets/meps-topics-access-to-health-care.md,data_rescue_project,MEPS Topics Access to Health Care,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp?topicid=1Z-1,None,2025-03-18,False,None,Health & Healthcare,1,MEPS Topics Access to Health Care,448,https://www.datalumos.org/datalumos/project/222581/version/V1/view,PDF,Finished,0.000,2025-03-03,"DRP, DL",None,www.datalumos.org,10.3886/E222581V1,222581,V1,10.3886/E222581V1
905,_datasets/meps-topics-childrens-health.md,data_rescue_project,MEPS Topics Children's Health,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp?topicid=2Z-1,None,2025-03-18,False,None,Health & Healthcare,1,MEPS Topics Children's Health,449,https://www.datalumos.org/datalumos/project/222581/version/V1/view,PDF,Finished,0.000,2025-03-03,"DRP, DL",None,www.datalumos.org,10.3886/E222581V1,222581,V1,10.3886/E222581V1
906,_datasets/meps-topics-childrens-insurance-coverage.md,data_rescue_project,MEPS Topics Children's Insurance Coverage,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp?topicid=3Z-1,None,2025-03-18,False,None,Health & Healthcare,1,MEPS Topics Children's Insurance Coverage,450,https://www.datalumos.org/datalumos/project/222581/version/V1/view,PDF,Finished,0.000,2025-03-03,"DRP, DL",None,www.datalumos.org,10.3886/E222581V1,222581,V1,10.3886/E222581V1
907,_datasets/meps-topics-dental-visitsuseevents-and-expenditures.md,data_rescue_project,MEPS Topics Dental Visits/Use/Events and Expenditures,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp?topicid=47Z-1,None,2025-03-18,False,None,Health & Healthcare,1,MEPS Topics Dental Visits/Use/Events and Expenditures,451,https://www.datalumos.org/datalumos/project/222581/version/V1/view,PDF,Finished,0.000,2025-03-03,"DRP, DL",None,www.datalumos.org,10.3886/E222581V1,222581,V1,10.3886/E222581V1
908,_datasets/meps-topics-disability.md,data_rescue_project,MEPS Topics Disability,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp?topicid=20Z-1,None,2025-03-18,False,None,Health & Healthcare,1,MEPS Topics Disability,452,https://www.datalumos.org/datalumos/project/222581/version/V1/view,PDF,Finished,0.000,2025-03-03,"DRP, DL",None,www.datalumos.org,10.3886/E222581V1,222581,V1,10.3886/E222581V1
909,_datasets/meps-topics-doctor-visitsuseevents-and-expenditures.md,data_rescue_project,MEPS Topics Doctor Visits/Use/Events and Expenditures,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp?topicid=21Z-1,None,2025-03-18,False,None,Health & Healthcare,1,MEPS Topics Doctor Visits/Use/Events and Expenditures,453,https://www.datalumos.org/datalumos/project/222581/

metadata

In [46]:
import time
import requests
import pandas as pd

MAILTO = "maja.murawka@student.uva.nl"   # recommended
session = requests.Session()
session.headers.update({
    "Accept": "application/vnd.api+json",
    "User-Agent": f"metadata-harvest/1.0 (mailto:{MAILTO})"
})


rows = []
for doi in unique_dois:
    url = f"https://api.datacite.org/dois/{doi}"
    try:
        r = session.get(url, timeout=30)

        # basic backoff if rate limited
        if r.status_code == 429:
            time.sleep(2)
            r = session.get(url, timeout=30)

        if r.status_code != 200:
            rows.append({"doi": doi, "dc_ok": False, "dc_status": r.status_code, "dc_error": r.text[:200]})
            continue

        data = r.json()
        attrs = data.get("data", {}).get("attributes", {})
        rel = data.get("data", {}).get("relationships", {})

        # keep raw JSON (handy for later)
        row = {
            "doi": doi,
            "dc_ok": True,
            "dc_status": 200,
            "dc_raw": data,  # you can drop later if it's too big
        }

        # --- common scalar fields ---
        for k in [
            "prefix", "suffix", "publisher", "publicationYear", "language",
            "version", "url", "contentUrl", "schemaVersion", "source",
            "isActive", "state", "reason", "metadataVersion",
            "created", "registered", "published", "updated",
            "viewCount", "downloadCount", "referenceCount", "citationCount",
            "partCount", "partOfCount", "versionCount", "versionOfCount"
        ]:
            row[f"dc_{k}"] = attrs.get(k)

        # --- types (nested) ---
        types = attrs.get("types") or {}
        row["dc_types_ris"] = types.get("ris")
        row["dc_types_bibtex"] = types.get("bibtex")
        row["dc_types_citeproc"] = types.get("citeproc")
        row["dc_types_schemaOrg"] = types.get("schemaOrg")
        row["dc_types_resourceTypeGeneral"] = types.get("resourceTypeGeneral")

        # --- identifiers lists ---
        row["dc_identifiers"] = attrs.get("identifiers")
        row["dc_alternateIdentifiers"] = attrs.get("alternateIdentifiers")

        # --- titles ---
        titles = attrs.get("titles") or []
        row["dc_title"] = titles[0].get("title") if titles else None
        row["dc_titles_all"] = titles

        # --- creators ---
        creators = attrs.get("creators") or []
        row["dc_creators_all"] = creators
        row["dc_creators_names"] = "; ".join([c.get("name","") for c in creators if c.get("name")]) or None
        row["dc_creators_types"] = "; ".join([c.get("nameType","") for c in creators if c.get("nameType")]) or None

        # --- contributors ---
        contributors = attrs.get("contributors") or []
        row["dc_contributors_all"] = contributors
        row["dc_contributors_names"] = "; ".join([c.get("name","") for c in contributors if c.get("name")]) or None

        # --- subjects ---
        subjects = attrs.get("subjects") or []
        row["dc_subjects_all"] = subjects
        row["dc_subjects"] = "; ".join([s.get("subject","") for s in subjects if s.get("subject")]) or None

        # --- dates ---
        dates = attrs.get("dates") or []
        row["dc_dates_all"] = dates
        # handy “picked” fields
        collected = [d.get("date") for d in dates if (d.get("dateType") or "").lower() == "collected"]
        issued = [d.get("date") for d in dates if (d.get("dateType") or "").lower() == "issued"]
        row["dc_date_collected"] = collected[0] if collected else None
        row["dc_date_issued"] = issued[0] if issued else None

        # --- descriptions ---
        descs = attrs.get("descriptions") or []
        row["dc_descriptions_all"] = descs
        abstract = [d.get("description") for d in descs if (d.get("descriptionType") or "").lower() == "abstract"]
        row["dc_abstract"] = abstract[0] if abstract else None

        # --- geoLocations ---
        geos = attrs.get("geoLocations") or []
        row["dc_geoLocations_all"] = geos
        row["dc_geo_places"] = "; ".join(
            [g.get("geoLocationPlace","") for g in geos if g.get("geoLocationPlace")]
        ) or None

        # --- rights & funding ---
        row["dc_rightsList"] = attrs.get("rightsList")
        row["dc_fundingReferences"] = attrs.get("fundingReferences")

        # --- related identifiers ---
        relids = attrs.get("relatedIdentifiers") or []
        row["dc_relatedIdentifiers_all"] = relids
        row["dc_isVersionOf"] = "; ".join(
            [x.get("relatedIdentifier","") for x in relids if (x.get("relationType") or "") == "IsVersionOf"]
        ) or None

        # --- relationships (structured links) ---
        # e.g. provider/client/versionOf ids
        def rel_id(name):
            d = rel.get(name, {}).get("data")
            if isinstance(d, dict):
                return d.get("id")
            if isinstance(d, list):
                return "; ".join([x.get("id","") for x in d if isinstance(x, dict) and x.get("id")]) or None
            return None

        row["dc_rel_client"] = rel_id("client")
        row["dc_rel_provider"] = rel_id("provider")
        row["dc_rel_media"] = rel_id("media")
        row["dc_rel_versionOf"] = rel_id("versionOf")
        row["dc_rel_versions"] = rel_id("versions")
        row["dc_rel_parts"] = rel_id("parts")
        row["dc_rel_partOf"] = rel_id("partOf")
        row["dc_rel_citations"] = rel_id("citations")
        row["dc_rel_references"] = rel_id("references")

        rows.append(row)

    except Exception as e:
        rows.append({"doi": doi, "dc_ok": False, "dc_status": None, "dc_error": str(e)})

datacite_df = pd.DataFrame(rows)
datacite_df.head()

doi  dc_ok  dc_status  \
0  10.3886/e226824v1   True        200   
1  10.3886/e223443v1   True        200   
2  10.3886/e222881v1   True        200   
3  10.3886/e222043v1   True        200   
4  10.3886/e222044v2   True        200   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [51]:
df["doi_norm"] = (
    df["doi"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({"nan": None, "none": None, "": None})
)

datacite_df["doi_norm"] = (
    datacite_df["doi"]
    .astype(str)
    .str.strip()
    .str.lower()
)

df = df.merge(
    datacite_df.drop(columns=["doi"], errors="ignore"),
    on="doi_norm",
    how="left"
)

In [52]:
df

file  \
0                                             _datasets/10j-injunctions.md   
1                                     _datasets/2002-2024-mbda-grantees.md   
2                                    _datasets/2006-iur-public-database.md   
3                                _datasets/2014-minority-veteran-report.md   
4           _datasets/2016-americorps-mes-americorps-member-exit-survey.md   
...                                                                    ...   
1760                            _datasets/your-right-to-federal-records.md   
1761  _datasets/youth-risk-behavior-surveillance-system-yrbss-2023-2025.md   
1762            _datasets/youth-risk-behavior-surveillance-system-yrbss.md   
1763            _datasets/youth-risk-behavior-surveillance-system-yrbss.md   
1764                              _datasets/zambia-usaid-education-data.md   

                   schema  \
0     data_rescue_project   
1     data_rescue_project   
2     data_rescue_project   
3     data_rescue_project   
4     data_rescue_project   
...                   ...   
1760  data_rescue_project   
1761  data_rescue_project   
1762  data_rescue_project   
1763  data_rescue_project   
1764  data_rescue_project   

                                                          title  \
0                                             10(j) Injunctions   
1                                       2002-2024 MBDA Grantees   
2                                      2006 IUR Public Database   
3                                  2014 Minority Veteran Report   
4             2016 AmeriCorps MES AmeriCorps Member Exit Survey   
...                                                         ...   
1760                              Your Right to Federal Records   
1761  Youth Risk Behavior Surveillance System (YRBSS) 2023-2025   
1762            Youth Risk Behavior Surveillance System (YRBSS)   
1763            Youth Risk Behavior Surveillance System (YRBSS)   
1764                                Zambia USAID Education Data   

                                                                   organization  \
0                                                National Labor Relations Board   
1                                          Minority Business Development Agency   
2                                               Environmental Protection Agency   
3     Office of Information and Technology - IT Operations and Services (ITOPS)   
4                                                                    AmeriCorps   
...                                                                         ...   
1760                                 Federal Mediation and Conciliation Service   
1761                           Centers for Disease Control and Prevention (CDC)   
1762                           Centers for Disease Control and Prevention (CDC)   
1763                           Centers for Disease Control and Prevention (CDC)   
1764                               M/CIO Development Information Solution (DIS)   

                                          agency             websites  \
0                 National Labor Relations Board             nlrb.gov   
1                         Department of Commerce             mbda.gov   
2                Environmental Protection Agency              epa.gov   
3                 Department of Veterans Affairs          data.va.gov   
4                                     AmeriCorps  data.americorps.gov   
...                                          ...                  ...   
1760  Federal Mediation and Conciliation Service             fmcs.gov   
1761     Department of Health and Human Services              cdc.gov   
1762     Department of Health and Human Services              cdc.gov   
1763     Department of Health and Human Services              cdc.gov   
1764   U.S. Agency for International Development       data.usaid.gov   

                                                                                                         

In [53]:
no_doi = df[df["doi"].isna()]

no_doi

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count,project_title,id,url,format,status,size,download_date,maintainer,notes,domain,doi,project_id,version,doi_guess,dc_ok_x,dc_status_x,dc_raw_x,dc_prefix_x,dc_suffix_x,dc_publisher_x,dc_publicationYear_x,dc_language_x,dc_version_x,dc_url_x,dc_contentUrl_x,dc_schemaVersion_x,dc_source_x,dc_isActive_x,dc_state_x,dc_reason_x,dc_metadataVersion_x,dc_created_x,dc_registered_x,dc_published_x,dc_updated_x,dc_viewCount_x,dc_downloadCount_x,dc_referenceCount_x,dc_citationCount_x,dc_partCount_x,dc_partOfCount_x,dc_versionCount_x,dc_versionOfCount_x,dc_types_ris_x,dc_types_bibtex_x,dc_types_citeproc_x,dc_types_schemaOrg_x,dc_types_resourceTypeGeneral_x,dc_identifiers_x,dc_alternateIdentifiers_x,dc_title_x,dc_titles_all_x,dc_creators_all_x,dc_creators_names_x,dc_creators_types_x,dc_contributors_all_x,dc_contributors_names_x,dc_subjects_all_x,dc_subjects_x,dc_dates_all_x,dc_date_collected_x,dc_date_issued_x,dc_descriptions_all_x,dc_abstract_x,dc_geoLocations_all_x,dc_geo_places_x,dc_rightsList_x,dc_fundingReferences_x,dc_relatedIdentifiers_all_x,dc_isVersionOf_x,dc_rel_client_x,dc_rel_provider_x,dc_rel_media_x,dc_rel_versionOf_x,dc_rel_versions_x,dc_rel_parts_x,dc_rel_partOf_x,dc_rel_citations_x,dc_rel_references_x,dc_error_x,doi_norm,dc_ok_y,dc_status_y,dc_raw_y,dc_prefix_y,dc_suffix_y,dc_publisher_y,dc_publicationYear_y,dc_language_y,dc_version_y,dc_url_y,dc_contentUrl_y,dc_schemaVersion_y,dc_source_y,dc_isActive_y,dc_state_y,dc_reason_y,dc_metadataVersion_y,dc_created_y,dc_registered_y,dc_published_y,dc_updated_y,dc_viewCount_y,dc_downloadCount_y,dc_referenceCount_y,dc_citationCount_y,dc_partCount_y,dc_partOfCount_y,dc_versionCount_y,dc_versionOfCount_y,dc_types_ris_y,dc_types_bibtex_y,dc_types_citeproc_y,dc_types_schemaOrg_y,dc_types_resourceTypeGeneral_y,dc_identifiers_y,dc_alternateIdentifiers_y,dc_title_y,dc_titles_all_y,dc_creators_all_y,dc_creators_names_y,dc_creators_types_y,dc_contributors_all_y,dc_contributors_names_y,dc_subjects_all_y,dc_subjects_y,dc_dates_all_y,dc_date_collected_y,dc_date_issued_y,dc_descriptions_all_y,dc_abstract_y,dc_geoLocations_all_y,dc_geo_places_y,dc_rightsList_y,dc_fundingReferences_y,dc_relatedIdentifiers_all_y,dc_isVersionOf_y,dc_rel_client_y,dc_rel_provider_y,dc_rel_media_y,dc_rel_versionOf_y,dc_rel_versions_y,dc_rel_parts_y,dc_rel_partOf_y,dc_rel_citations_y,dc_rel_references_y,dc_error_y
2,_datasets/2006-iur-public-database.md,data_rescue_project,2006 IUR Public Database,Environmental Protection Agency,Environmental Protection Agency,epa.gov,https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database,None,2025-09-18,True,https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database,Climate & Environment,1,2006 IUR Public Database,1281,https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi10.7910/DVN/F3A62W,"mhtml, ZIP, PDF",Finished,0.005,2025-02-26,"HD, CAFE-RCC","There was a 2006 version and a 2002-1986 version. Both are archived, hence 2 dataverse URLs. ~ag. Seperate Metadata https://www.epa.gov/chemical-data-reporting/summary-cdr-reporting-requirements-year, https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database",dataverse.harvard.edu,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,_datasets/administrative-law-judge-decisions.md,data_rescue_project,Administrative Law Judge Decisions,Nati

In [57]:
df_2 = df[df["doi"].notna()].copy()

In [58]:
df_2.isna().mean().sort_values(ascending=False).head(50)

dc_rel_citations_x                1.000000
dc_reason_y                       1.000000
dc_contentUrl_x                   1.000000
dc_rel_versions_x                 1.000000
dc_rel_parts_x                    1.000000
dc_rel_partOf_x                   1.000000
dc_rel_references_x               1.000000
dc_contentUrl_y                   1.000000
dc_reason_x                       1.000000
dc_rel_parts_y                    1.000000
dc_rel_references_y               1.000000
dc_rel_partOf_y                   1.000000
dc_rel_versions_y                 1.000000
dc_error_x                        0.999271
dc_error_y                        0.998541
description                       0.997812
dc_contributors_names_y           0.995624
dc_contributors_names_x           0.995624
dc_geo_places_x                   0.983224
dc_date_collected_x               0.981036
dc_subjects_x                     0.978118
metadata_url                      0.975201
dc_language_x                     0.974471
dc_isVersio

In [60]:
threshold = 0.97

cols_to_drop = df_2.columns[
    df_2.isna().mean() >= threshold
]

df_2 = df_2.drop(columns=cols_to_drop)

print(f"Dropped {len(cols_to_drop)} columns")

Dropped 74 columns


In [61]:
df_2.columns

Index(['file', 'schema', 'title', 'organization', 'agency', 'websites',
       'data_source', 'last_modified', 'metadata_available', 'category',
       'resource_count', 'project_title', 'id', 'url', 'format', 'status',
       'size', 'download_date', 'maintainer', 'notes', 'domain', 'doi',
       'project_id', 'version', 'doi_guess', 'dc_ok_x', 'dc_status_x',
       'doi_norm', 'dc_ok_y', 'dc_status_y', 'dc_raw_y', 'dc_prefix_y',
       'dc_suffix_y', 'dc_publisher_y', 'dc_publicationYear_y',
       'dc_language_y', 'dc_version_y', 'dc_url_y', 'dc_schemaVersion_y',
       'dc_source_y', 'dc_isActive_y', 'dc_state_y', 'dc_metadataVersion_y',
       'dc_created_y', 'dc_registered_y', 'dc_published_y', 'dc_updated_y',
       'dc_viewCount_y', 'dc_downloadCount_y', 'dc_referenceCount_y',
       'dc_citationCount_y', 'dc_partCount_y', 'dc_partOfCount_y',
       'dc_versionCount_y', 'dc_versionOfCount_y', 'dc_types_ris_y',
       'dc_types_bibtex_y', 'dc_types_citeproc_y', 'dc_types_schemaO

In [62]:
df_2.to_csv('drp_withdoi.csv')